In [1]:
# Fix randomness and hide warnings
seed = 42
%load_ext autoreload
%autoreload 2

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

# Fix randomness and hide warnings
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

import matplotlib.pyplot as plt
plt.rc('font', size=16)

2.15.0


In [2]:
input_folder = '../data/training_dataset/'
training_data = np.load(input_folder + 'training_data.npy')
valid_periods = np.load(input_folder + 'valid_periods.npy')
categories = np.load(input_folder + 'categories.npy')

In [3]:
from C2.notebooks.Utils import split_dataset, build_sequences_optimized

dataset, labels = build_sequences_optimized(training_data, valid_periods)
train_validation_data, train_validation_labels, test_data, test_labels = split_dataset(dataset, labels)
train_data, train_labels, validation_data, validation_labels = split_dataset(train_validation_data, train_validation_labels)

train_data = train_data.reshape((train_data.shape[0], train_data.shape[1], 1))
train_labels = train_labels.reshape((train_labels.shape[0], train_labels.shape[1], 1))
test_data = test_data.reshape((test_data.shape[0], test_data.shape[1], 1))
test_labels = test_labels.reshape((test_labels.shape[0], test_labels.shape[1], 1))
test_data = test_data.reshape((test_data.shape[0], test_data.shape[1], 1))
test_labels = test_labels.reshape((test_labels.shape[0], test_labels.shape[1], 1))
train_data.shape

Dataset shape:  (48000, 200)


In [4]:
def augment_data(train_data, train_labels, num_rounds=3):
    num_augmentations=3
    total_size = num_rounds * num_augmentations * len(train_data)

    # Preallocate numpy arrays
    augmented_train_data = np.empty((total_size, *train_data.shape[1:]))
    augmented_train_labels = np.empty((total_size, *train_labels.shape[1:]))

    for round in range(num_rounds):
        print("Augmentation round: ", round)
        for i in range(len(train_data)):
            # Calculate the start index for this round and data point
            start_idx = round * num_augmentations * len(train_data) + i * num_augmentations

            # Add noise
            noise = np.random.normal(0, 0.05, train_data[i].shape)
            augmented_train_data[start_idx] = train_data[i] + noise
            augmented_train_labels[start_idx] = train_labels[i]

            # Add scaling
            scaling = np.random.uniform(0.8, 1.2)
            augmented_train_data[start_idx + 1] = train_data[i] * scaling
            augmented_train_labels[start_idx + 1] = train_labels[i] * scaling

            # Add constant value
            constant = np.random.uniform(-0.1, 0.1)
            augmented_train_data[start_idx + 2] = train_data[i] + constant
            augmented_train_labels[start_idx + 2] = train_labels[i] + constant

    return augmented_train_data, augmented_train_labels

augmented_train_data, augmented_train_labels = augment_data(train_data, train_labels)
# augmented_train_data, augmented_train_labels = augment_data(dataset, labels)
augmented_train_data = augmented_train_data.reshape((augmented_train_data.shape[0], augmented_train_data.shape[1], 1))
augmented_train_labels = augmented_train_labels.reshape((augmented_train_labels.shape[0], augmented_train_labels.shape[1], 1))

# del train_data, train_labels

Augmentation round:  0
Augmentation round:  1
Augmentation round:  2


In [18]:
def build_CONV_LSTM_model(input_shape, output_shape):
    assert input_shape[0] >= output_shape[0], "Input time steps should be >= output time steps"

    # Input layer
    input_layer = tf.keras.layers.Input(shape=input_shape, name='input_layer')

    # LSTM layers
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True, name='lstm'))(input_layer)
    x = tf.keras.layers.Dropout(0.2)(x)
    # Convolutional layers
    x = tf.keras.layers.Conv1D(12, 5, padding='same', activation='relu', name='conv1')(x)
    x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)  # Adding MaxPooling to downsample

    # Another Convolutional layer
    x = tf.keras.layers.Conv1D(12, 3, padding='same', activation='relu', name='conv2')(x)

    # Output layer
    output_layer = tf.keras.layers.Conv1D(output_shape[1], 3, padding='same', name='output_layer')(x)

    # Cropping to match output shape
    crop_size = output_layer.shape[1] - output_shape[0]
    output_layer = tf.keras.layers.Cropping1D((0, crop_size), name='cropping')(output_layer)

    # Model instantiation
    model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer, name='Better_CONV_LSTM_model')

    # Compiling the model
    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                  metrics=[tf.keras.metrics.RootMeanSquaredError()],
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

    return model


def build_LSTM_model(input_shape, output_shape):
    assert input_shape[0] >= output_shape[0], "Input time steps should be >= output time steps"

    # Input layer
    input_layer = tf.keras.layers.Input(shape=input_shape, name='input_layer')

    # LSTM layers
    
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True, name='lstm'))(input_layer)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, name='lstm'))(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    # x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    # Output layer
    output_layer = tf.keras.layers.Dense(output_shape[0], name='output_layer', activation='sigmoid')(x)
    
    # Model instantiation
    model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer, name='Better_CONV_LSTM_model')

    # Compiling the model
    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                  metrics=[tf.keras.metrics.RootMeanSquaredError()],
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

    return model

def build_LSTM_ENC_DEC_model(input_shape, output_shape):
    assert input_shape[0] >= output_shape[0], "Input time steps should be >= output time steps"

    # Input layer
    input_layer = tf.keras.layers.Input(shape=input_shape, name='input_layer')

    # LSTM layers
    
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, name='lstm'))(input_layer)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.RepeatVector(output_shape[0])(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, name='lstm', return_sequences=True))(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Flatten()(x)
    flatten_input = tf.keras.layers.Flatten()(input_layer)
    # Skipper layer
    x = tf.keras.layers.Concatenate()([x, flatten_input])
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    # Output layer
    output_layer = tf.keras.layers.Dense(output_shape[0], name='output_layer', activation='sigmoid')(x)
    # Model instantiation
    model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer, name='Better_CONV_LSTM_model')

    # Compiling the model
    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                  metrics=[tf.keras.metrics.MeanAbsoluteError()],
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

    return model

In [19]:
input_shape = augmented_train_data.shape[1:]
output_shape = augmented_train_labels.shape[1:]
batch_size = 256
epochs = 10

model = build_LSTM_ENC_DEC_model(input_shape, output_shape)
model.summary()
tfk.utils.plot_model(model, expand_nested=True, show_shapes=True) 

Model: "Better_CONV_LSTM_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_layer (InputLayer)    [(None, 200, 1)]             0         []                            
                                                                                                  
 bidirectional_6 (Bidirecti  (None, 64)                   8704      ['input_layer[0][0]']         
 onal)                                                                                            
                                                                                                  
 dropout_6 (Dropout)         (None, 64)                   0         ['bidirectional_6[0][0]']     
                                                                                                  
 repeat_vector_3 (RepeatVec  (None, 18, 64)               0         ['dropout

In [20]:
# Train the model
history = model.fit(
    x = augmented_train_data,
    y = augmented_train_labels,
    batch_size = batch_size,
    epochs = epochs,
    # validation_data=(validation_data, validation_labels),
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', min_delta=0.0003, patience=12, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=8, factor=0.1, min_lr=1e-5)
    ]
).history

Epoch 1/10
1688/1688 [==============================] - 41s 20ms/step - loss: 0.0231 - mean_absolute_error: 0.1097 - lr: 0.0010
Epoch 2/10
1688/1688 [==============================] - 34s 20ms/step - loss: 0.0204 - mean_absolute_error: 0.1019 - lr: 0.0010
Epoch 3/10
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0196 - mean_absolute_error: 0.0995 - lr: 0.0010
Epoch 4/10
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0191 - mean_absolute_error: 0.0981 - lr: 0.0010
Epoch 5/10
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0186 - mean_absolute_error: 0.0967 - lr: 0.0010
Epoch 6/10
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0182 - mean_absolute_error: 0.0955 - lr: 0.0010
Epoch 7/10
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0177 - mean_absolute_error: 0.0943 - lr: 0.0010
Epoch 8/10
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0174 - mean_absolute_err

In [21]:
model.save('../models/ENC_DEC_model_light_deep.h5')

In [18]:
from C2.notebooks import Utils
from importlib import reload
reload(Utils)
test_data_predictions = model.predict(test_data)
Utils.evaluate_model(test_data_predictions, test_labels)

300/300 [==============================] - 4s 7ms/step
MAE: 0.11244506225454622
MSE: 0.026357403430591904
RMSE: 0.16234963329367857


In [ ]:
# Evaluate random samples
Utils.evaluate_model(np.random.rand(*test_data_predictions.shape), test_labels)

In [ ]:
for _ in range(10):
    idx = np.random.randint(0, len(test_data))
    Utils.plot_predictions(test_data, test_data_predictions, test_labels,idx)
    # Utils.plot_predictions(train_data, test_data_predictions, train_labels,idx)

In [14]:
test_data_predictions = model.predict(augmented_train_data[:100])


4/4 [==============================] - 2s 16ms/step


In [17]:
test_data_predictions[:,:9]


array([[0.2704957 , 0.1888153 , 0.18665417, 0.1464724 , 0.14875363,
        0.19473699, 0.15720256, 0.16792174, 0.13143562],
       [0.2836118 , 0.18262929, 0.17019786, 0.14843903, 0.15006319,
        0.19932815, 0.16695595, 0.171206  , 0.1366302 ],
       [0.28594938, 0.19818988, 0.18856847, 0.16557342, 0.16527791,
        0.20994113, 0.18197347, 0.18599871, 0.15453036],
       [0.27936515, 0.23412623, 0.23680733, 0.23384032, 0.2451307 ,
        0.37971997, 0.31790653, 0.2536183 , 0.25412613],
       [0.24417631, 0.18021816, 0.20076667, 0.19942991, 0.19388372,
        0.33033836, 0.2872646 , 0.21693026, 0.20889732],
       [0.33802056, 0.25613523, 0.278994  , 0.27566132, 0.26500046,
        0.42824548, 0.36959705, 0.28930038, 0.2817797 ],
       [0.40999848, 0.43533123, 0.47332177, 0.4547165 , 0.46077627,
        0.4264002 , 0.46418178, 0.42985392, 0.4723959 ],
       [0.50249714, 0.50151515, 0.5348998 , 0.5371747 , 0.5283224 ,
        0.5260282 , 0.5413178 , 0.5489111 , 0.5344858 ],
